In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
x_train_list = pickle.load(open('../gen_data/x_train--features2.ipynb--.pickle','rb'))
x_test_list = pickle.load(open('../gen_data/x_test--features2.ipynb--.pickle','rb'))

sales = pd.read_csv('../gen_data/train1.csv')

### Generate time specific market price data

In [3]:
mean_price = sales.groupby(['date_block_num','item_id'],as_index=True).item_price.mean()
std_price = sales.groupby(['date_block_num','item_id'],as_index=True).item_price.std(ddof=1)
median_price = sales.groupby(['date_block_num','item_id'],as_index=True).item_price.median()

price_data = pd.concat([mean_price,std_price,median_price],axis=1)
price_data.columns = ['mean_prevmonth_item_price','std_prevmonth_item_price','median_prevmonth_item_price']
price_data.reset_index(inplace=True)
price_data['date_block_num'] += 1

### Generate shop-time price data

In [4]:
#not gonna worry about getting median price by transaction, will 
#settle for median price by row (which could incnlude many transactions)
shop_item_data = sales.groupby(['date_block_num','shop_id','item_id'],as_index=False).item_price.median()
shop_item_data['date_block_num'] += 1
shop_item_data.columns = ['date_block_num','shop_id','item_id','median_prevmonth_shop_item_price']
shop_item_data.head()

,date_block_num,shop_id,item_id,median_prevmonth_shop_item_price
0,1,2,27,2499.0
1,1,2,33,499.0
2,1,2,317,299.0
3,1,2,438,299.0
4,1,2,471,399.0


In [6]:
price_data = downcast(price_data)

In [7]:
def merge_price_data(df):
    return df.merge(price_data,how='left').merge(shop_item_data,how='left')

In [8]:
x_train_list = list(map(merge_price_data,x_train_list))
x_test_list = list(map(merge_price_data,x_test_list))

In [9]:
def add_prop_median(df):
    df['prop_median_item_price'] = df.median_prevmonth_shop_item_price/df.median_prevmonth_item_price
    return df

In [10]:
x_train_list = list(map(add_prop_median,x_train_list))
x_test_list = list(map(add_prop_median,x_test_list))

### Save

In [15]:
pickle.dump(x_train_list,open('../gen_data/x_train--features3.ipynb--.pickle','wb'))
pickle.dump(x_test_list,open('../gen_data/x_test--features3.ipynb--.pickle','wb'))